In [1]:
# readImage() 서버로 이미지 받아오기
  
def sendInfo():
    cursor = data_db.cursor()
    sql = 'INSERT INTO gojiseo(date, electUse, waterUse, electPrice, waterPrice, publicPrice, totalPrice) values(date_this, elec_use, water_use, elec_price, water_price, public_price, total_price'
    
def pieInfo():
    cursor = data_db.cursor()
    sql = 'INSERT INTO pieData(eletPrice, waterPrice, etcIndividualPrice, publicPrice, total_price) values(elec_price, wat_price, etc_indivi_price, public_price, total_price)'
    
def readUserData():
    cursor = data_db.cursor()
    sql = 'SELECT electPrice, waterPrice, totalPrice FROM userData'

def sendUserPredict():
    cursor = data_db.cursor()
    sql = 'INSERT INTO userPredict(total_predict, rateOfChange) values(user_data_predict[0], user_data_predict[1])'
    
    
# readDanjiImform() 지역코드, 아파트이름 받아오기
# api는 시간 많이 걸리니까 메시지 출력하기 ('정보를 가져오고 있습니다.')

def sendDanjiComparison():
    cursor = data_db.cursor()
    sql = 'INSERT INTO danjiUserPredict(date, danjiPrice, userPredict) values(danji_predict(sidoCode, aptName)[0],danji_predict(sidoCode, aptName)[1],danji_predict(sidoCode, aptName)[2])'